# Implementation of Mathematics Pipeline

In [ ]:
!pip install langchain-groq
!pip install openai
!pip install langchain-google-genai
!pip install langchain-core 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import inspect

# Processing the AIMesaage and converting it to human Message

from langchain_core.messages.ai import AIMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [ ]:
google_api_key="<----Your API Key---->"
groq_api_key="<----Your API Key---->"
model_name="llama3-70b-8192"
#model_name="gemini-pro"

In [ ]:
model = ChatGroq(temperature=0.7, 
                groq_api_key=groq_api_key, 
                model_name=model_name,
                max_tokens=8192,
                model_kwargs={"top_p": 0.7,#"json_response": False
                             },
                )

#print(inspect.getsource(ChatGroq))

In [ ]:
from langchain_google_genai import GoogleGenerativeAI, HarmBlockThreshold, HarmCategory

model = GoogleGenerativeAI(
    model=model_name,
    google_api_key=google_api_key,
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

## First chain to extact the ``Steps to Solve the problem``

In [ ]:
system=" ###Context### \
You are an AI assistant that helps people solve Mathematics and Statistics. \
### Instruction ### \
 1. Provide List of the Steps to solve the question. Like` Step 1:,Step 2:` . \
 Do not perform caculations and get do not get into details."

In [ ]:
question="Q: \
Consider the following dataset for a binary classification problem in $\mathbb{R}^{2}$ \
\[x_{1}=[\begin{matrix}1\\ 0\end{matrix}],y_{1}=+1\]\
\[x_{2}=[\begin{matrix}0\\ 1\end{matrix}],y_{2}=+1\]\
\[x_{3}=[\begin{matrix}-1\\ 0\end{matrix}],y_{3}=-1\]\
\[x_{4}=[\begin{matrix}0\\ -1\end{matrix}],y_{4}=-1\]\
Choose all linear classifiers that result in zero misclassifications on this dataset. Here, w is the weight vector for the linear classifier.\
A)  \[w=[\begin{matrix}1\\ 2\end{matrix}]\]\
B)  \[w=[\begin{matrix}10\\ 19\end{matrix}]\]\
C) \[w=[\begin{matrix}-1\\ -4\end{matrix}]\]\
D) \[w=[\begin{matrix}-5\\ 3\end{matrix}]\]\
E) \[w=[\begin{matrix}5\\ -3\end{matrix}]\]"

In [ ]:
human = "{content}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
output_parser = StrOutputParser()

In [ ]:
chain = prompt | model #| output_parser
output=await chain.ainvoke({"content": question }) ### Outputs AI message
print(f'########## Model used : {model_name} #########')
print(output.content)

#print(output)

## Second chain to provide ``Steps as input to Solve the problem and Solution in the form of code``

In [ ]:
steps = HumanMessage(content=question +" \n "+ output.content) ### Converts ai message to human message

In [ ]:
print(steps.content)

In [ ]:
input = steps.content

In [ ]:
system=" ###Context### \
You are an AI assistant that helps people solve Mathematics and Statistics. \
### Instruction ### \
 Provide the Python code to solve the Questions from the given instructions. \
 Provide comment for each step in the code."

In [ ]:
human = "{content}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

prompt_value = prompt.invoke({"content": input })


chain = prompt | model
code=await chain.ainvoke({"content": input })
print(f'########## Model used : {model_name} #########')
print(code.content)
#print(code)

In [ ]:
import numpy as np

# Define the data points and labels
x1 = np.array([1, 0])
y1 = 1
x2 = np.array([0, 1])
y2 = 1
x3 = np.array([-1, 0])
y3 = -1
x4 = np.array([0, -1])
y4 = -1

# Define the options for the weight vectors
options = [
    np.array([1, 2]),
    np.array([10, 19]),
    np.array([-1, -4]),
    np.array([-5, 3]),
    np.array([5, -3])
]

# Initialize a list to store the correct options
correct_options = []

# Iterate over the options
for w in options:
    # Initialize a flag to check if the option results in zero misclassifications
    zero_misclassifications = True
    
    # Iterate over the data points
    for x, y in [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]:
        # Calculate the sign of w^T*x
        sign = np.sign(np.dot(w, x))
        
        # Check if the sign matches the label
        if sign != y:
            zero_misclassifications = False
            break
    
    # If the option results in zero misclassifications, add it to the correct options
    if zero_misclassifications:
        correct_options.append(w)

# Print the correct options
print("The correct options are:")
for w in correct_options:
    print(w)

### Step 3 Proving the input to third chain for latex calculation.

In [ ]:
input = human_message.content + "\n Solution : "

print(f'::::::::::::::::: Below is sample input to the chain ::::::::::::')
print(input)

In [ ]:
system=" ###Context### \
You are an AI assistant that helps people solve Mathematics and Statistics. \
### Instruction ### \
 Provide the latex code to solve the Questions from the given instructions. \
 Use code interpreter to perform calculations while solving the questions. \
 Use the steps provided in the question to provide the solution. \
### Output ### \
Only provide the latex document in the output. \
Please include the dependencies used by the latex cod in \\usepackage. \
"

In [ ]:
human = "{content}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

prompt_value = prompt.invoke({"content": input })


chain = prompt | model
content=await chain.ainvoke({"content": input })
print(f'########## Model used : {model_name} #########')
print(content.content)